install deps

In [1]:
# Install unsloth library
# Please restart session when asked to restart session.
# Rerun this code to make sure that all libraries are installed.

#core
!pip install --no-deps bitsandbytes accelerate xformers==0.0.29 peft trl triton


#unsloth and training
!pip install --no-deps cut_cross_entropy unsloth_zoo
!pip install unsloth wandb

#for datasets and tokenizers
!pip install sentencepiece protobuf datasets huggingface_hub hf_transfer
!pip install pandas

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


load the qwen model

In [2]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 2048
dtype = None
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/Llama-3.2-1B",
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit
)
chat_model = model
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ],
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",  # True or "unsloth" for very long context
    random_state=3407,
    use_rslora=False,
    loftq_config=None,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


/home/tlsto/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.3.19: Fast Llama patching. Transformers: 4.50.3.
   \\   /|    NVIDIA GeForce RTX 3090. Num GPUs = 1. Max memory: 24.0 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 8.6. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth 2025.3.19 patched 16 layers with 16 QKV layers, 16 O layers and 16 MLP layers.


write a test csv trainingdata file to the colab dir




Ask it a question here

In [5]:

question = "Help! I accidentally attached my hand to the shed with a nail gun!"

prompt_style = """<|begin_of_text|><|start_header_id|>system<|end_header_id|>
<|eot_id|><|start_header_id|>user<|end_header_id|>
{}<|eot_id|><|start_header_id|>assistant<|end_header_id|>
"""
print("tesst!")
FastLanguageModel.for_inference(model)  # Unsloth has 2x faster inference!
inputs = tokenizer(question, return_tensors="pt").to("cuda")

outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=1200,
    use_cache=True,
)
response = tokenizer.batch_decode(outputs)
print(response)
answer_sep = "<|eot_id|><|start_header_id|>assistant<|end_header_id|>"
print(f"Prompt:\n{question}")
print(f"\n\nResponse:")
print(response
#print(response[0].split(answer_sep)[1].replace("<|eot_id|>",""))

tesst!
['<|begin_of_text|>Help! I accidentally attached my hand to the shed with a nail gun! Now I can’t get it off!\nIf you’ve been in the business of selling hand tools, you know that there is a certain level of care and attention to detail that goes into every piece of equipment. That is why it’s important to protect your tools, especially if you use them for a living.\xa0\nIt’s important to keep your tools safe and sound, but what if you use them for a living? You might be tempted to use tools that you have already used, or you might be tempted to use tools that you have already used. You might be tempted to use tools that you have already used, or you might be tempted to use tools that you have already used.\nWhen you use tools that you have already used, you might be tempted to use tools that you have already used. You might be tempted to use tools that you have already used. You might be tempted to use tools that you have already used. You might be tempted to use tools that you 

Save Model

In [6]:
model.save_pretrained_gguf("dir", tokenizer, quantization_method = "q8_0")

Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 17.99 out of 31.3 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


100%|███████████████████████████████████████████████████████████████████████████████████| 16/16 [00:00<00:00, 45.30it/s]


Unsloth: Saving tokenizer... Done.
Done.


Unsloth: Converting llama model. Can use fast conversion = False.


==((====))==  Unsloth: Conversion from QLoRA to GGUF information
   \\   /|    [0] Installing llama.cpp might take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF 16bits might take 3 minutes.
\        /    [2] Converting GGUF 16bits to ['q8_0'] might take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: Installing llama.cpp. This might take 3 minutes...


RuntimeError: Unsloth: The file ('llama.cpp/llama-quantize' or 'llama.cpp/llama-quantize.exe' if you are on Windows WSL) or 'llama.cpp/quantize' does not exist.
But we expect this file to exist! Maybe the llama.cpp developers changed the name or check extension of the llama-quantize file.

In [ ]:
!pip install -qU langchain-huggingface
!pip install -qU langchain-community
!pip install faiss-cpu
!pip install llama-cpp-python

Load dataset and LLM

In [ ]:
from langchain_community.document_loaders import TextLoader
from langchain_community.llms import LlamaCpp

data = '/content/rag_doc.txt'
desired_context_length = 4096

loader = TextLoader(data)

docs = loader.load()
llm = LlamaCpp(model_path="/content/dir/unsloth.Q8_0.gguf", n_ctx=desired_context_length)

In [ ]:
print(docs[0].metadata)
print(docs[0].page_content[0:500])

Embedding Model

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings



text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200,separators=['New Patient:'])
splits = text_splitter.split_documents(docs)

embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")


Vector Store

In [ ]:
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS
import faiss

embedding_dim = len(embedding_model.embed_query("hello world"))
index = faiss.IndexFlatL2(embedding_dim)

vector_store = FAISS(
    embedding_function=embedding_model,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)


vector_store.add_texts([d.page_content for d in splits])
retriever = vector_store.as_retriever(search_type="mmr", search_kwargs={"k": 5})

Create Chain

In [ ]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate


system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know."
    "\n\n"
    "{context}"
)


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

Ask Questions Here

In [ ]:
results = rag_chain.invoke({"input": "Treatment plan for degloving injury."})
print(results['answer'])